Model Training and validation (RSNA dataset)

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pydicom
import numpy as np
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torchvision.models import VGG16_Weights
from sklearn.model_selection import train_test_split

# Random seed 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

class PulmonaryEmbolismDataset(Dataset):
    def __init__(self, dataframe, root_dir, window_center, window_width, transform=None):
        self.data_frame = dataframe
        self.root_dir = root_dir
        self.window_center = window_center
        self.window_width = window_width
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        study_uid = self.data_frame.iloc[idx]['StudyInstanceUID']
        series_uid = self.data_frame.iloc[idx]['SeriesInstanceUID']
        sop_uid = self.data_frame.iloc[idx]['SOPInstanceUID']
        file_path = os.path.join(self.root_dir, study_uid, series_uid, f"{sop_uid}.dcm")
        dicom_image, _ = self.load_and_process_dicom(file_path)
        if self.transform:
            dicom_image = self.transform(dicom_image)
        label = self.data_frame.iloc[idx]['pe_present_on_image']
        return dicom_image, label, file_path

    def load_and_process_dicom(self, dicom_file):
        ds = pydicom.dcmread(dicom_file)
        image = ds.pixel_array
        if "RescaleIntercept" in ds and "RescaleSlope" in ds:
            intercept = ds.RescaleIntercept
            slope = ds.RescaleSlope
            hu_image = self.convert_to_hounsfield_units(image, intercept, slope)
        else:
            hu_image = image.astype(np.int16)
        windowed_image = self.apply_windowing_correctly(hu_image, self.window_center, self.window_width)
        resized_image = Image.fromarray(windowed_image).resize((224, 224))
        image_tensor = torch.tensor(np.array(resized_image), dtype=torch.float32).unsqueeze(0)
        image_tensor = image_tensor.repeat(3, 1, 1)  
        image_tensor = image_tensor / 255.0  
        return image_tensor, hu_image

    def apply_windowing_correctly(self, image, window_center, window_width):
        window_min = window_center - window_width / 2
        window_max = window_center + window_width / 2
        windowed_image = np.clip(image, window_min, window_max)
        return ((windowed_image - window_min) / (window_max - window_min) * 255).astype(np.uint8)

    def convert_to_hounsfield_units(self, image, intercept, slope):
        return image * slope + intercept

# Define transformations
transform = transforms.Compose([])  # No transformation

# Load the dataset
csv_file = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/RSNA dataset/rsna-str-pulmonary-embolism-detection/train_processed.csv'
root_dir = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/RSNA dataset/rsna-str-pulmonary-embolism-detection/train'
df = pd.read_csv(csv_file)

# Sample the dataset
sampled_df = df.groupby('StudyInstanceUID').apply(lambda x: x.sample(frac=0.1, random_state=42)).reset_index(drop=True)

# Split the sampled dataset by StudyInstanceUID to avoid data leakage
unique_studies = sampled_df['StudyInstanceUID'].unique()
train_studies, temp_studies = train_test_split(unique_studies, test_size=0.2, random_state=42)
val_studies, test_studies = train_test_split(temp_studies, test_size=0.5, random_state=42)

train_df = sampled_df[sampled_df['StudyInstanceUID'].isin(train_studies)].reset_index(drop=True)
val_df = sampled_df[sampled_df['StudyInstanceUID'].isin(val_studies)].reset_index(drop=True)
test_df = sampled_df[sampled_df['StudyInstanceUID'].isin(test_studies)].reset_index(drop=True)

# Create the datasets
train_dataset = PulmonaryEmbolismDataset(train_df, root_dir, window_center=40, window_width=400, transform=transform)
val_dataset = PulmonaryEmbolismDataset(val_df, root_dir, window_center=40, window_width=400, transform=transform)
test_dataset = PulmonaryEmbolismDataset(test_df, root_dir, window_center=40, window_width=400, transform=transform)

# Calculate the percentage of pe_label=1 in each dataset
train_pe_label_percent = (train_df['pe_present_on_image'].mean()) * 100
val_pe_label_percent = (val_df['pe_present_on_image'].mean()) * 100
test_pe_label_percent = (test_df['pe_present_on_image'].mean()) * 100

print(f"Train dataset: {train_pe_label_percent:.2f}% of samples have pe_label=1")
print(f"Validation dataset: {val_pe_label_percent:.2f}% of samples have pe_label=1")
print(f"Test dataset: {test_pe_label_percent:.2f}% of samples have pe_label=1")

print("dataset load complete.")

# Create data loaders
batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 1)

model.load_state_dict(torch.load('VGG16_best_model_weights.pth'))

model = nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
model = model.cuda()

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

print("model initialization.")

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    best_auc = 0.0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_preds = []
        train_labels = []

        for inputs, labels, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            train_preds.append(outputs.detach().cpu().numpy())
            train_labels.append(labels.detach().cpu().numpy())

        epoch_loss = running_loss / len(train_loader.dataset)
        train_preds = np.concatenate(train_preds)
        train_labels = np.concatenate(train_labels)
        train_auc = roc_auc_score(train_labels, train_preds)

        model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for inputs, labels, _ in val_loader:
                inputs = inputs.cuda()
                labels = labels.cuda().float().unsqueeze(1)
                outputs = model(inputs)
                val_preds.append(outputs.cpu().numpy())
                val_labels.append(labels.cpu().numpy())

        val_preds = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Train AUC: {train_auc:.3f}, Val AUC: {val_auc:.3f}')
        torch.save(model.module.state_dict(), 'VGG16_model_weights.pth')

        if val_auc > best_auc:
            best_auc = val_auc
            best_model_wts = model.module.state_dict()  # Save the model.module state_dict for DataParallel

    model.module.load_state_dict(best_model_wts)

    return model, best_auc

# Train the model
model, best_val_auc = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)
print(f'Best Val AUC: {best_val_auc:.3f}')

# Save the best model weights
torch.save(model.module.state_dict(), 'VGG16_best_model_weights.pth')

# Plot ROC curves
def plot_roc_curve(loader, model, dataset_type):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic ({dataset_type} Set)')
    plt.legend(loc="lower right")
    plt.show()

    return roc_auc

# Plot ROC curve for train set
train_auc = plot_roc_curve(train_loader, model, 'Train')
print(f'Train AUC: {train_auc:.3f}')

# Plot ROC curve for validation set
val_auc = plot_roc_curve(val_loader, model, 'Validation')
print(f'Validation AUC: {val_auc:.3f}')

# Load the best model weights for testing
model.module.load_state_dict(torch.load('VGG16_best_model_weights.pth'))

# Evaluate on the test set
test_auc = plot_roc_curve(test_loader, model, 'Test')
print(f'Test AUC: {test_auc:.3f}')

# Calculate additional metrics for the test set
def evaluate_metrics(loader, model, threshold=0.5):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    preds_binary = (all_preds >= threshold).astype(int)

    tp = (preds_binary * all_labels).sum()
    tn = ((1 - preds_binary) * (1 - all_labels)).sum()
    fp = (preds_binary * (1 - all_labels)).sum()
    fn = ((1 - preds_binary) * all_labels).sum()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)

    return sensitivity, specificity, accuracy, ppv, npv

sensitivity, specificity, accuracy, ppv, npv = evaluate_metrics(test_loader, model)

print(f'Test Set Metrics:')
print(f'Sensitivity: {sensitivity:.3f}')
print(f'Specificity: {specificity:.3f}')
print(f'Accuracy: {accuracy:.3f}')
print(f'PPV: {ppv:.3f}')
print(f'NPV: {npv:.3f}')


Model testing (RSNA dataset)

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pydicom
import numpy as np
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torchvision.models import VGG16_Weights
from sklearn.model_selection import train_test_split

# Random seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Plot ROC curves
def plot_roc_curve(loader, model, dataset_type):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic ({dataset_type} Set)')
    plt.legend(loc="lower right")
    plt.show()

    return roc_auc

class PulmonaryEmbolismDataset(Dataset):
    def __init__(self, dataframe, root_dir, window_center, window_width, transform=None):
        self.data_frame = dataframe
        self.root_dir = root_dir
        self.window_center = window_center
        self.window_width = window_width
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        study_uid = self.data_frame.iloc[idx]['StudyInstanceUID']
        series_uid = self.data_frame.iloc[idx]['SeriesInstanceUID']
        sop_uid = self.data_frame.iloc[idx]['SOPInstanceUID']
        file_path = os.path.join(self.root_dir, study_uid, series_uid, f"{sop_uid}.dcm")
        dicom_image, _ = self.load_and_process_dicom(file_path)
        if self.transform:
            dicom_image = self.transform(dicom_image)
        label = self.data_frame.iloc[idx]['pe_present_on_image']
        return dicom_image, label, file_path

    def load_and_process_dicom(self, dicom_file):
        ds = pydicom.dcmread(dicom_file)
        image = ds.pixel_array
        if "RescaleIntercept" in ds and "RescaleSlope" in ds:
            intercept = ds.RescaleIntercept
            slope = ds.RescaleSlope
            hu_image = self.convert_to_hounsfield_units(image, intercept, slope)
        else:
            hu_image = image.astype(np.int16)
        windowed_image = self.apply_windowing_correctly(hu_image, self.window_center, self.window_width)
        resized_image = Image.fromarray(windowed_image).resize((224, 224))
        image_tensor = torch.tensor(np.array(resized_image), dtype=torch.float32).unsqueeze(0)
        image_tensor = image_tensor.repeat(3, 1, 1)  # 3 채널로 복사
        image_tensor = image_tensor / 255.0  # 0~1 범위로 정규화
        return image_tensor, hu_image

    def apply_windowing_correctly(self, image, window_center, window_width):
        window_min = window_center - window_width / 2
        window_max = window_center + window_width / 2
        windowed_image = np.clip(image, window_min, window_max)
        return ((windowed_image - window_min) / (window_max - window_min) * 255).astype(np.uint8)

    def convert_to_hounsfield_units(self, image, intercept, slope):
        return image * slope + intercept

# Define transformations
transform = transforms.Compose([])  # No transformation

# Load the dataset
csv_file = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/RSNA dataset/rsna-str-pulmonary-embolism-detection/train_processed.csv'
root_dir = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/RSNA dataset/rsna-str-pulmonary-embolism-detection/train'
df = pd.read_csv(csv_file)

# Sample the dataset
sampled_df = df.groupby('StudyInstanceUID').apply(lambda x: x.sample(frac=0.1, random_state=42)).reset_index(drop=True)

# Split the sampled dataset by StudyInstanceUID to avoid data leakage
unique_studies = sampled_df['StudyInstanceUID'].unique()
train_studies, temp_studies = train_test_split(unique_studies, test_size=0.2, random_state=42)
val_studies, test_studies = train_test_split(temp_studies, test_size=0.5, random_state=42)

train_df = sampled_df[sampled_df['StudyInstanceUID'].isin(train_studies)].reset_index(drop=True)
val_df = sampled_df[sampled_df['StudyInstanceUID'].isin(val_studies)].reset_index(drop=True)
test_df = sampled_df[sampled_df['StudyInstanceUID'].isin(test_studies)].reset_index(drop=True)

# Create the datasets
test_dataset = PulmonaryEmbolismDataset(test_df, root_dir, window_center=40, window_width=400, transform=transform)
test_pe_label_percent = (test_df['pe_present_on_image'].mean()) * 100
print(f"Test dataset: {test_pe_label_percent:.2f}% of samples have pe_label=1")


print("dataset load complete.")

# Create data loaders
batch_size = 2048

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 1)

model.load_state_dict(torch.load('VGG16_best_model_weights.pth'))

model = nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
model = model.cuda()

# Evaluate on the test set
test_auc = plot_roc_curve(test_loader, model, 'Test')
print(f'Test AUC: {test_auc:.3f}')

# Calculate additional metrics for the test set
def evaluate_metrics(loader, model, threshold=0.5):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    preds_binary = (all_preds >= threshold).astype(int)

    tp = (preds_binary * all_labels).sum()
    tn = ((1 - preds_binary) * (1 - all_labels)).sum()
    fp = (preds_binary * (1 - all_labels)).sum()
    fn = ((1 - preds_binary) * all_labels).sum()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)

    return sensitivity, specificity, accuracy, ppv, npv

sensitivity, specificity, accuracy, ppv, npv = evaluate_metrics(test_loader, model)

print(f'Test Set Metrics:')
print(f'Sensitivity: {sensitivity:.3f}')
print(f'Specificity: {specificity:.3f}')
print(f'Accuracy: {accuracy:.3f}')
print(f'PPV: {ppv:.3f}')
print(f'NPV: {npv:.3f}')

Inference in external validation set

In [ ]:
# REAL CONTRAST KNU EXTERNAL N=62 

import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torchvision.models import VGG16_Weights

# Random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Plot ROC curves
def plot_roc_curve(loader, model, dataset_type):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic ({dataset_type} Set)')
    plt.legend(loc="lower right")
    plt.savefig('TEST IN KNU FULL APPEND (REAL CONTRAST)', format='eps', dpi=300)

    plt.show()

    return roc_auc

class PulmonaryEmbolismDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data_frame = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        study_uid = self.data_frame.iloc[idx]['StudyInstanceUID']
        series_uid = self.data_frame.iloc[idx]['SeriesInstanceUID']
        sop_uid = str(self.data_frame.iloc[idx]['SOPInstanceUID']).zfill(4)  # Ensure SOPInstanceUID is a 4-digit string
        file_path = os.path.join(self.root_dir, study_uid, series_uid, f"{sop_uid}.jpg")
        image = Image.open(file_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.data_frame.iloc[idx]['pe_present_on_image']
        return image, label, file_path

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the dataset
csv_file = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/output with pe label (GAN lc, 0.5)_KNU_append+KNUnormal.csv'
root_dir = '/workspace/kyt3426/project_chest_CT_GAN/external_data_jpg_KNUnormal_and_append'
df = pd.read_csv(csv_file)

# Create the datasets using the entire dataframe for external validation
test_dataset = PulmonaryEmbolismDataset(df, root_dir, transform=transform)
test_pe_label_percent = (df['pe_present_on_image'].mean()) * 100
print(f"Test dataset: {test_pe_label_percent:.2f}% of samples have pe_label=1")

# Print additional statistics
num_slices = len(df)
num_slices_pe_0 = len(df[df['pe_present_on_image'] == 0])
num_slices_pe_1 = len(df[df['pe_present_on_image'] == 1])
num_unique_patients = df['StudyInstanceUID'].nunique()

print(f"Total number of slices: {num_slices}")
print(f"Number of slices with label 0: {num_slices_pe_0}")
print(f"Number of slices with label 1: {num_slices_pe_1}")
print(f"Number of patients: {num_unique_patients}")

print("Dataset load complete.")

# Create data loaders
batch_size = 256

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 1)

model.load_state_dict(torch.load('VGG16_best_model_weights.pth'))

model = nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
model = model.cuda()

# Evaluate on the test set
test_auc = plot_roc_curve(test_loader, model, 'Test')
print(f'Test AUC: {test_auc:.3f}')



In [ ]:
# GAN CONTRAST KNU EXTERNAL N=62

import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torchvision.models import VGG16_Weights

# Random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Plot ROC curves
def plot_roc_curve(loader, model, dataset_type):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in loader:
            inputs = inputs.cuda()
            labels = labels.cuda().float().unsqueeze(1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic ({dataset_type} Set)')
    plt.legend(loc="lower right")
    plt.savefig('TEST IN KNU FULL APPEND (GAN CONTRAST)', format='eps', dpi=300)

    plt.show()

    return roc_auc

class PulmonaryEmbolismDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data_frame = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        study_uid = self.data_frame.iloc[idx]['StudyInstanceUID']
        series_uid = self.data_frame.iloc[idx]['SeriesInstanceUID']
        sop_uid = str(self.data_frame.iloc[idx]['SOPInstanceUID']).zfill(4)  # Ensure SOPInstanceUID is a 4-digit string
        file_path = os.path.join(self.root_dir, study_uid, series_uid, f"{sop_uid}.jpg")
        image = Image.open(file_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.data_frame.iloc[idx]['pe_present_on_image']
        return image, label, file_path

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the dataset
csv_file = '/workspace/kyt3426/project_chest_CT_GAN/Embolism_classification_(RSNA_dataset)/output with pe label (GAN lc, 0.5)_KNU_append+KNUnormal.csv'
root_dir = '/workspace/kyt3426/project_chest_CT_GAN/Inference_test_based_on_dualCT_model_only_disease_lc0.5_KNU_EXTERNAL'
df = pd.read_csv(csv_file)

# Create the datasets using the entire dataframe for external validation
test_dataset = PulmonaryEmbolismDataset(df, root_dir, transform=transform)
test_pe_label_percent = (df['pe_present_on_image'].mean()) * 100
print(f"Test dataset: {test_pe_label_percent:.2f}% of samples have pe_label=1")

# Print additional statistics
num_slices = len(df)
num_slices_pe_0 = len(df[df['pe_present_on_image'] == 0])
num_slices_pe_1 = len(df[df['pe_present_on_image'] == 1])
num_unique_patients = df['StudyInstanceUID'].nunique()

print(f"Total number of slices: {num_slices}")
print(f"Number of slices with label 0: {num_slices_pe_0}")
print(f"Number of slices with label 1: {num_slices_pe_1}")
print(f"Number of patients: {num_unique_patients}")

print("Dataset load complete.")

# Create data loaders
batch_size = 256

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 1)

model.load_state_dict(torch.load('VGG16_best_model_weights.pth'))

model = nn.DataParallel(model, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
model = model.cuda()

# Evaluate on the test set
test_auc = plot_roc_curve(test_loader, model, 'Test')
print(f'Test AUC: {test_auc:.3f}')

